In [58]:
import os
from functools import reduce

file  = "hyperparameter.txt"
with open(file,"r") as f:
    config = [hp.split(":") for hp in  f.readlines()]
config = {hp[0]:hp[1].strip("\n").split(",") for hp in config}

output_file = file.split(".")[0] + "_output.txt"

total_hp = list(config.keys())
total_hp_len = [len(item) for _,item in config.items()]
total_hp_comb = reduce(lambda a,b: a*b, total_hp_len)

template = " ".join(["--" + hp + " {}" for hp in total_hp])
    
with open(output_file, "w") as f:
    for num in range(total_hp_comb):
        now_index = []
        div = total_hp_comb
        remain_num = num
        for cur_len in total_hp_len:
            div = div// cur_len
            now_index.append(remain_num // div)
            remain_num = remain_num % div
        now_comb = template.format(*[config[total_hp[hp_index]][value_index] for hp_index,value_index in enumerate(now_index)])
        f.write(now_comb + "\n")

In [1]:
from time import sleep

In [3]:
import os
from functools import reduce

In [2]:
import random


In [1]:
import os
import subprocess
import time
from datetime import datetime
from termcolor import colored
import logging

exp_name = "apple:" + datetime.now().strftime("%m-%d-%Y_%H:%M:%S")
if not os.path.exists("logs"):
    os.mkdir("logs")
    
os.mkdir(os.path.join("logs",exp_name))

logging.basicConfig(filename=os.path.join("./logs",exp_name,"log.log"),
                    filemode='w',
                    datefmt='%H:%M:%S',
                   level=logging.DEBUG,
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s')


console = logging.StreamHandler()
console.setLevel(logging.INFO)
# set a format which is simpler for console use
formatter = logging.Formatter('%(name)-12s: %(levelname)-8s %(message)s')
# tell the handler to use this format
console.setFormatter(formatter)
# add the handler to the root logger
logger = logging.getLogger()
logger.addHandler(console)


logger.info("Running " + exp_name)

gpu_list = ["0","1","2"]
max_jobs_per_gpu = 2
script = "test.py"
file  = "hyperparameter_output.txt"


gpu_tasks = {gpu:[] for gpu in gpu_list}
with open(file,"r") as f:
    config = [hp.strip("\n") for hp in f.readlines()]
total_exp_num = len(config)
    
failed_exp = []
finised_exp = 0
while finised_exp < total_exp_num:
    # find empty gpu
    use_gpu = None
    for gpu in gpu_list:
        if len(gpu_tasks[gpu]) < max_jobs_per_gpu:
            use_gpu = gpu
            break
            
    if use_gpu and len(config) > 0:
        # run with gpu
        run_command = "python {} --gpu {} {}".format(script,use_gpu,config.pop())
        logger.info(" ({}/{}) ".format(finised_exp,total_exp_num) +
              " start to execute: " + run_command)
        proc = subprocess.Popen(run_command.split())
        gpu_tasks[use_gpu].append((proc,run_command))
        
    else:
        for _, tasks in gpu_tasks.items():
            finised_jobs = []
            for job_id, (proc,run_command) in enumerate(tasks):
                
                try:
                    proc.communicate(timeout=1)
                except:
                    continue
                    
                if proc.returncode is not None:
                    if proc.returncode == 0:
                        logger.info(" ({}/{}) ".format(finised_exp,total_exp_num) +
                              " The exp {} is finished".format(run_command))
                    else:
                        logger.warning(" ({}/{}) ".format(finised_exp,total_exp_num) +
                              " The exp {} is ".format(run_command) + "failed")
                        failed_exp.append(run_command)
                        
                finised_jobs.append(job_id)
                finised_exp += 1
            
            for job_id in sorted(finised_jobs, reverse=True):
                del tasks[job_id]
        
        time.sleep(1)
        
logger.info(exp_name+" is finished!")
        
failed_exp = [" ".join(exp.split()[4:]) for exp in failed_exp]
with open(os.path.join("logs",exp_name,"failed_exp.txt"),"w") as f:
    for exp in failed_exp:
        f.write(exp)
        f.write("\n")
                    

root        : INFO     Running apple:10-09-2022_22:46:17
root        : INFO      (0/10)  start to execute: python test.py --gpu 0 
root        : INFO      (0/10)  start to execute: python test.py --gpu 0 --epoch 1 --rep 3 --bz 3
root        : INFO      (0/10)  start to execute: python test.py --gpu 1 --epoch 1 --rep 3 --bz 2
root        : INFO      (0/10)  start to execute: python test.py --gpu 1 --epoch 1 --rep 3 --bz 1
root        : INFO      (0/10)  start to execute: python test.py --gpu 2 --epoch 1 --rep 2 --bz 3
root        : INFO      (0/10)  start to execute: python test.py --gpu 2 --epoch 1 --rep 2 --bz 2
Traceback (most recent call last):
  File "test.py", line 20, in <module>
    assert random.randint(0,10) >= 2
AssertionError
root        : INFO      (0/10)  The exp python test.py --gpu 1 --epoch 1 --rep 3 --bz 1 is finished
root        : INFO      (1/10)  The exp python test.py --gpu 2 --epoch 1 --rep 2 --bz 3 is finished
root        : INFO      (2/10)  The exp python test.p